In [1]:
import os, time
import sys
sys.path.append("")
sys.path.append("../..")
import unittest
import hashlib
import numpy as np
from alg.FedPC import pc, pc_voting
from causallearn.search.ConstraintBased import PC
from causallearn.utils.cit import chisq, fisherz, gsq, kci, mv_fisherz
from causallearn.graph.SHD import SHD
from causallearn.utils.DAG2CPDAG import dag2cpdag
from causallearn.utils.TXT2GeneralGraph import txt2generalgraph
from causallearn.search.ScoreBased.GES import ges
import alg.fed_GES as fed_GES
from utils.utils import get_fl_data
import copy
import pandas as pd

from utils.Synthetic import *
import notears_admm.utils as admm_utils
from notears_admm.linear_admm import notears_linear_admm
from notears_admm.nonlinear_admm import notears_nonlinear_admm
from notears_admm.postprocess import postprocess
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import utils.utils as fed_utils

import torch
torch.set_default_dtype(torch.double)

sys.argv=['TestFedPC.py']
class HiddenPrints:
    def __init__(self, activated=True):
        # activated参数表示当前修饰类是否被激活
        self.activated = activated
        self.original_stdout = None

    def open(self):
        sys.stdout.close()
        sys.stdout = self.original_stdout

    def close(self):
        self.original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        # 这里的os.devnull实际上就是Linux系统中的“/dev/null”
        # /dev/null会使得发送到此目标的所有数据无效化，就像“被删除”一样
        # 这里使用/dev/null对sys.stdout输出流进行重定向

    def __enter__(self):
        if self.activated:
            self.close()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.activated:
            self.open()


/home/zhaoyu/anaconda3/envs/fed_pc/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
train_data_ls=[]
train_label_ls=[]
test_data_ls=[]
for i in range(3):
    with open(f'data/lucas/lucas{i}_train.data','rb') as f:
        raw_data = f.readlines()
    data = [str(item) for item in raw_data]
    train_data_ls.append(np.array([data[i][2:-3].strip().split() for i in range(len(data))],int))
    with open(f'data/lucas/lucas{i}_train.targets','rb') as f:
        raw_data = f.readlines()
    train_label_ls.append(np.array(raw_data,int))
    with open(f'data/lucas/lucas{i}_test.csv','rb') as f:
        raw_data = f.readlines()
    data = [str(item) for item in raw_data]
    data2 = [data[i][3:-4].split('" "') for i in range(len(data))]
    test_data_ls.append(np.array(data2[1:],int)[:,1:])

In [3]:
# data for casual discovery
with open('data/lucas/lucas_train.csv','rb') as f:
    raw_data = f.readlines()
data = [str(item) for item in raw_data]
data2 = [data[i][3:-4].split('" "') for i in range(len(data))]
col_name = data2[0]
data_value = np.array([[float(i) for i in row_data[1:]] for row_data in data2[1:]])
client_num=2
df=pd.DataFrame(data_value,columns=col_name)
df.to_csv('data/lucas/lucas_train_new.csv',index=False)

data2 = [data[i][3:-4].split('" "') for i in range(len(data))]
col_name = data2[0]
data_value = np.array([[float(i) for i in row_data[1:]] for row_data in data2[1:]])
client_num=2
df=pd.DataFrame(data_value,columns=col_name)
df.to_csv('data/lucas/lucas_train_new.csv',index=False)

In [5]:
for client_num in [8]:
    col_idx=[]
    method_ls=['None']
    col_idx.append([True for i in range(data_value.shape[1]-1)])
    # fed_pc
    cg = pc(data_value, 0.05, 'fed_cit', True, 0, -1, node_names=None, client_num=client_num)
    col_idx.append((cg.G.graph[:,-1]!=0)[:-1])
    method_ls.append('fed_pc')
    #notears_admm
    X,_ = fed_utils.read_csv('data/lucas/lucas_train_new.csv')
    n=X.shape[0]//client_num;d=X.shape[1]    
    Xs = X[:n*client_num].reshape(client_num, n, d)
    B_est = notears_nonlinear_admm(Xs, lambda1=0.01, verbose=False)
    B_processed = postprocess(B_est, threshold=0.05)
    col_idx.append(((B_processed+B_processed.T)[:,-1]!=0)[:-1])
    method_ls.append('notears_admm')
    # rfcd
    with HiddenPrints():
        cg = fed_GES.ges(data_value, score_func='local_score_BDeu', maxP=None, parameters=None,client_num=client_num)
    col_idx.append((cg['G'].graph[:,-1]!=0)[:-1])
    method_ls.append('rfcd')
    #pc_voting
    fl_data = fed_utils.get_fl_data(data_value,client_num)
    cg = pc_voting(fl_data, client_num,cg['G'],data_value.shape[1])
    col_idx.append((cg.graph[:,-1]!=0)[:-1])
    method_ls.append('pc_voting')
    # pc_cit_voting
    cg = pc(data_value, 0.05, 'cit_with_voting', True, 0, -1, node_names=None, client_num=client_num,is_iid = True)
    col_idx.append((cg.G.graph[:,-1]!=0)[:-1])
    method_ls.append('pc_cit_voting')

    for i in range(3):
        for j in range(len(method_ls)):
            for alpha in np.arange(0,2,0.1):
                x_train = train_data_ls[i][:,col_idx[j]]
                y_train = train_label_ls[i]
                model1 = Ridge(alpha=alpha)
                model1.fit(x_train,y_train)
                y_pred1 = model1.predict(x_train)
                y_pred2 = model1.predict(test_data_ls[i][:,:-1][:,col_idx[j]])
                print(f'dataset_num:{i} alpha:{alpha} client_num:{client_num}  method:{method_ls[j]}\n\
                    train MSE:{mean_squared_error(y_pred1,y_train):.3f}\t\
                        test MSE:{mean_squared_error(y_pred2,test_data_ls[0][:,-1]):.3f}')
                #print('feature_num:',sum(np.array(col_idx[j])),'\t',np.array(col_idx[j]))
                print('feature_num:',sum(np.array(col_idx[j])))#,'\t',np.array(col_idx[j]))
                
                print('\n')

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

dataset_num:0 alpha:0.0 client_num:8  method:None
                    train MSE:0.423	                        test MSE:0.271
feature_num: 11


dataset_num:0 alpha:0.1 client_num:8  method:None
                    train MSE:0.423	                        test MSE:0.271
feature_num: 11


dataset_num:0 alpha:0.2 client_num:8  method:None
                    train MSE:0.423	                        test MSE:0.271
feature_num: 11


dataset_num:0 alpha:0.30000000000000004 client_num:8  method:None
                    train MSE:0.423	                        test MSE:0.271
feature_num: 11


dataset_num:0 alpha:0.4 client_num:8  method:None
                    train MSE:0.423	                        test MSE:0.271
feature_num: 11


dataset_num:0 alpha:0.5 client_num:8  method:None
                    train MSE:0.423	                        test MSE:0.271
feature_num: 11


dataset_num:0 alpha:0.6000000000000001 client_num:8  method:None
                    train MSE:0.423	                        t